## Problem
investpy offers multiple available exchanges and currencies to select from when downloading historical data from investing.com. The problem occurs when the module selects which exchange and currency to download the data from, and in some instances was incorrect. The data needs verifying the ensure the currency is now correct (verified against the Freetrade list). 

Workaround has been implemented by swapping the column names _symbol_ and _name_.

In [74]:
import investpy
import matplotlib.pyplot as plt

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [75]:
# load available etfs from Primary
etfs = context.catalog.load('etfs')

2021-01-27 22:06:23,057 - kedro.io.data_catalog - INFO - Loading data from `etfs` (CSVDataSet)...


In [76]:
# load downloaded etf historic data
etfs_hist = context.catalog.load('etfs_historical')

2021-01-27 22:06:27,895 - kedro.io.data_catalog - INFO - Loading data from `etfs_historical` (PartitionedDataSet)...


In [77]:
# loop through each etf in list of available etfs and read in the historical price data

summary = {}
match = []
for i, row in etfs.iterrows():
    filename = f"etf_{row.symbol_ft}_{row['isin']}"
    df = etfs_hist[filename]()
    first_row = df.iloc[0]
    years = 2021 - int(first_row['Date'][:4])
    currency = first_row['Currency']
    currency_match = currency == row['currency']
    exchange = first_row['Exchange']
    last_row = df.iloc[-1]
    latest_price = last_row['Close']
    match.append(currency_match)
    print(f"{row.symbol_ft} ({row.symbol}): age={years}yrs, currency={currency}, match={currency_match}, exchange={exchange}, latest_price={latest_price}, isin={row['isin']}")

UKRE (iShares MSCI Target UK Real Estate): age=6yrs, currency=GBP, match=True, exchange=London, latest_price=475.2, isin=IE00BRHZ0398
S100 (Source FTSE 100 UCITS): age=12yrs, currency=GBP, match=True, exchange=London, latest_price=6009.5, isin=IE00B60SWT88
IBTM (iShares $ Treasury Bond 7-10Yr UCITS): age=15yrs, currency=GBP, match=True, exchange=London, latest_price=159.19, isin=IE00B1FZS798
VEMT (Vanguard USD EM Government Bond): age=5yrs, currency=GBP, match=True, exchange=London, latest_price=38.22, isin=IE00BZ163L38
TRXS (Invesco US Treasury Bond 7-10 Year UCITS GBP Hedge): age=2yrs, currency=GBP, match=True, exchange=London, latest_price=4453.0, isin=IE00BF2FNB90
SLXX (iShares Core GBP Corporate Bond UCITS): age=17yrs, currency=GBP, match=True, exchange=London, latest_price=159.55, isin=IE00B00FV011
VUCP (Vanguard USD Corp Bd): age=5yrs, currency=GBP, match=True, exchange=London, latest_price=42.38, isin=IE00BZ163K21
IHHG (iShares $ High Yield Corp Bond UCITS GBP Hedged (D): age=3

ISF (iShares Core FTSE 100 UCITS): age=21yrs, currency=GBP, match=True, exchange=London, latest_price=646.6, isin=IE0005042456
IEER (iShares MSCI Eastern Europe Capped UCITS): age=16yrs, currency=GBP, match=True, exchange=London, latest_price=1704.0, isin=IE00B0M63953
MIDD (iShares FTSE 250 UCITS): age=17yrs, currency=GBP, match=True, exchange=London, latest_price=1921.0, isin=IE00B00FV128
VEVE (Vanguard FTSE Dev World): age=7yrs, currency=GBP, match=True, exchange=London, latest_price=58.17, isin=IE00BKX55T58
IWDP (iShares Developed Markets Property Yield UCITS): age=15yrs, currency=GBP, match=True, exchange=London, latest_price=1808.8, isin=IE00B1FZS350
VGER (Vanguard DAX UCITS): age=3yrs, currency=GBP, match=True, exchange=London, latest_price=22.56, isin=IE00BG143G97
VUKE (Vanguard FTSE 100 UCITS GBP Inc): age=9yrs, currency=GBP, match=True, exchange=London, latest_price=28.99, isin=IE00B810Q511
VWRL (Vanguard FTSE All-World UCITS USD Inc): age=9yrs, currency=GBP, match=True, excha

XUKX (db x-trackers FTSE 100 UCITS DR Income): age=14yrs, currency=GBP, match=True, exchange=London, latest_price=664.85, isin=LU0292097234
XXSC (db x-trackers MSCI Europe Small Cap UCITS DR 1C): age=13yrs, currency=GBP, match=True, exchange=London, latest_price=4519.5, isin=LU0322253906
TIPH (Lyxor US TIPS DR Monthly Hedged D-GBP): age=4yrs, currency=GBP, match=True, exchange=London, latest_price=113.28, isin=LU1452600601
GLTS (SPDR Barclays Capital 1-5 Year Gilt): age=9yrs, currency=GBP, match=True, exchange=London, latest_price=51.88, isin=IE00B6YX5K17
STHS (PIMCO Short-Term High Yield Corporate Bond Source): age=5yrs, currency=GBP, match=True, exchange=London, latest_price=9.55, isin=IE00BYXVWC37
UKCO (SSgA SPDR Barclays Cap Sterling Corporate Bond): age=9yrs, currency=GBP, match=True, exchange=London, latest_price=66.9, isin=IE00B4694Z11
XGIG (db x-trackers II iBoxx Global Inflation-Linked UCI): age=9yrs, currency=GBP, match=True, exchange=London, latest_price=2792.0, isin=LU06410

In [78]:
# get value counts for where currencies match or not
Counter(match)

Counter({True: 179})

List of incorrect historic data, with wrong currency:
- IBTM
- VUCP
- SEMB
- CU31
- ERNU
- etc

Other observations:
- Sometimes the price is in GBP and sometimes its in pence

## Implement a workaround
Reviewing a handful of incorrect currencies, it looks as though is the symbol column name is changed to _name_ in the standing data, this might trick it into selecting the correct currency.

etf.csv was edited accordingly. _name_ and _symbol_ column names swapped.

In [61]:
# check what investpy returns
etfs_df = investpy.get_etfs()
etfs_df[etfs_df.country == 'united kingdom'].head()

,country,symbol,full_name,name,isin,asset_class,currency,stock_exchange,def_stock_exchange
8090,united kingdom,Amundi ETF S&P 500 UCITS,Amundi ETF S&P 500 UCITS,500U,LU1681049018,equity,USD,London,False
8091,united kingdom,Amundi MSCI Europe Banks UCITS,Amundi MSCI Europe Banks UCITS,CB5,FR0010688176,equity,GBP,London,False
8092,united kingdom,Amundi DAX UCITS,Amundi MSCI Germany UCITS,CG1,FR0010655712,equity,GBP,London,False
8093,united kingdom,Amundi ETF MSCI Spain UCITS,Amundi ETF MSCI Spain UCITS,CS1,FR0010655746,equity,GBP,London,False
8094,united kingdom,Amundi MSCI Brazil UCITS,Amundi MSCI Brazil UCITS,BRZ,FR0010821793,equity,USD,London,False


In [63]:
# get recent data for freetrade symbol IBTM (ISTB)
ibtm = investpy.get_etf_recent_data('ISTB', 'United Kingdom', stock_exchange='London')

In [67]:
# IBTM results show Currency in GBP and latest price reflects the current price in Freetrade
ibtm.tail(1)

,Open,High,Low,Close,Volume,Currency,Exchange
Date,,,,,,,
2021-01-27,158.47,159.66,158.31,159.19,8588,GBP,London


In [68]:
# get recent data for Freetrade symbol IKOR 
ikor = investpy.get_etf_recent_data('IKOR', 'United Kingdom', stock_exchange='London')

In [69]:
# IKOR results shows Curerncy in GBP and latest price reflects the current price in Freetrade
ikor.tail(1)

,Open,High,Low,Close,Volume,Currency,Exchange
Date,,,,,,,
2021-01-27,4842.3,4843.8,4770.0,4777.5,42955,GBP,London
